In [6]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import json
import time

load_dotenv()

url = "https://api.newsedge.moodysanalytics.com/news"
headers = {
    "Authorization": os.getenv("M_TOKEN"),
    "accept": "application/json"
}

params = {
    "facets_stats_top": 1,
    "page_size": 25,
    "page_number": 1,
    "enable_stemming": "false",
    "enable_highlighting": "false",
    "query" : "Zomato"
}
r = requests.get(url=url, headers=headers, params=params)

print()
print(r)



<Response [200]>


In [7]:
f = pd.read_csv("./data/non us moody mapping - Sheet1.csv")
f = f[1000:]

In [8]:
print(f)

         ticker                                     name
1000    1812_JP                                   Kajima
1001    2802_JP                                Ajinomoto
1002    7186_JP                Concordia Financial Group
1003    2413_JP                                   M3 Inc
1004    3407_JP                              Asahi Kasei
...         ...                                      ...
1695  000001_CN                             Ping An Bank
1696     ADM_UK                            Admiral Group
1697     ABF_UK                 Associated British Foods
1698       7269                 Suzuki Motor Corporation
1699       ONGC  Oil and Natural Gas Corporation Limited

[700 rows x 2 columns]


In [9]:
data = {'db_ticker' : [], 'db_name':[], 'name':[],'orbis_id':[], 'id':[], 'symbol':[] }
# print(json.dumps(r.json()['data']['stories'][0]['organizations'], indent=2))

for db_ticker, db_name in zip(f['ticker'], f['name']):
    params["query"] = '"' + db_name + '"'
    r = requests.get(url=url, headers=headers, params=params)
    
    print(db_name)
    
    print(r.status_code)
    if(r.status_code != 200):
        print(r.reason, r.text)
        print(db_name)
        print()
        break
    try:
        orgs = r.json()['data']['stories'][0]['organizations']
    except (KeyError, IndexError) as e:
        print('e1', e, db_ticker)
        continue
    
    # print(json.dumps(orgs, indent=2))
    for data_value in orgs:
        try:
            data["db_ticker"].append(db_ticker)
            data["db_name"].append(db_name)
            data["name"].append(data_value['name'])
            data["orbis_id"].append(data_value['orbis_id'])
            data["id"].append(data_value['id'])
            try:
                data["symbol"].append(data_value['securities'][0]['symbol'])
            except IndexError:
                data["symbol"].append("")
        except (KeyError, IndexError) as e:
            print('e2', e, data_value)
    time.sleep(0.1)
    
print(*data, sep='\n')
for k, v in data.items():
    print(k, len(v))

Kajima
200
Ajinomoto
200
Concordia Financial Group
200
M3 Inc
200
Asahi Kasei
200
Shizuoka Financial Group
200
Dai Nippon Printing Co
200
YASKAWA Electric
200
Taisei
200
Mitsubishi Chemical Group
200
Obayashi
200
Tokyu Corporation
200
Daikin Industries
200
HCL
200
VAT Group
200
The Chiba Bank
200
Shimizu
200
Meiji Holdings
200
Sumitomo Metal Mining
200
Seiko Epson Corporation
200
OMRON
200
Fukuoka Financial Group
200
Hitachi Construction Machinery
200
Keisei Electric Railway Co.
200
Tokyo Electric Power Company
200
Kobe Steel
200
Tokyu Fudosan Holdings Corporation
200
Sojitz
200
Toto
200
Mitsui Chemicals
200
Oji Holdings
200
Sharp
200
Credit Saison Co
200
Mitsubishi Motors
200
Nissan Chemical
200
Sumitomo Chemical Company
200
The Yokohama Rubber
200
NGK Insulators
200
GMR Airports
200
HASEKO Corporation
200
Odakyu Electric Railway
200
JGC Holdings
200
Aozora Bank
200
NSK
200
Sumitomo Pharma
200
Okuma
200
Tokuyama Corporation
200
Mitsui Mining & Smelting
200
Takashimaya Company
200
Tobu

In [5]:
df = pd.DataFrame(data)
df.to_csv(r'.\data\extracted non us.csv2')



In [51]:
import os
print(os.getcwd())

C:\Users\Divya\PycharmProjects\VQ Scraper
